Importing necessary libraries

In [126]:
import pandas as pd

Map continent data with yield.csv

In [127]:
yield_data = pd.read_csv('processed_data/processed_yield.csv')

continent_data = pd.read_csv('processed_data/countries_by_continent_sorted.csv')

combined_yield = pd.merge(yield_data, continent_data, how='left', left_on='Area', right_on='Country')

combined_yield = combined_yield.drop(columns=['Country']).rename(columns={'Continent': 'Country_Continent'})

combined_yield.to_csv('processed_data/yield_continent.csv', index=False)

Map pesticides data with the updated yield

In [128]:
yield_data = pd.read_csv('processed_data/yield_continent.csv')

pesticides_data = pd.read_csv('processed_data/processed_pesticides.csv')

combined_yield = pd.merge(yield_data, pesticides_data, how='left', on=['Area', 'Year'])

combined_yield.to_csv('processed_data/yield_continent_pesticides.csv', index=False)


Map rainfall data with the updated yield

In [129]:
yield_data = pd.read_csv('processed_data/yield_continent_pesticides.csv')
 
rainfall_data = pd.read_csv('processed_data/processed_rainfall.csv')
 
combined_yield = pd.merge(yield_data, rainfall_data, how='left', on=['Area', 'Year'])
 
combined_yield.to_csv('processed_data/yield_continent_pesticides_rainfall.csv', index=False)


Map temperature data with the updated yield

In [130]:
yield_data = pd.read_csv('processed_data/yield_continent_pesticides_rainfall.csv')
 
temperature_data = pd.read_csv('processed_data/processed_temp.csv')

combined_yield = pd.merge(yield_data, temperature_data, how='left', left_on=['Area', 'Year'], right_on=['country', 'year'])

combined_yield = combined_yield.drop(columns=['country', 'year'])

combined_yield.to_csv('processed_data/combined_raw_data.csv', index=False)


Remove unecessary columns

In [131]:
file_path = 'processed_data/combined_raw_data.csv'
 
columns_to_delete = ['Domain Code', 'Domain_x', 'Area Code', 'Element_x', 'Element Code', 'Year Code', 'Domain_y', 'Element_y', ]

df = pd.read_csv(file_path)

df = df.drop(columns=columns_to_delete, errors='ignore')

df.to_csv(file_path, index=False)

delete rows that do not have values for the following columns 'Item_y', 'Unit_y', 'Value_y'

In [132]:

file_path = 'processed_data/combined_raw_data.csv'

columns_to_delete = ['Item_y', 'Unit_y', 'Value_y', 'Country_Continent']

df = pd.read_csv(file_path)

records_before = len(df)

df = df.dropna(subset=columns_to_delete, how='any')

records_after = len(df)

df.to_csv(file_path, index=False)
 
n = records_before - records_after
 
print(f"deleted: {n} records")


deleted: 4180 records


count the number of records per continent

In [133]:
file_path = 'processed_data/combined_raw_data.csv'
df = pd.read_csv(file_path)

result = df.groupby(['Country_Continent', 'Item_x']).size().reset_index(name='Count')

print(result)


   Country_Continent                Item_x  Count
0             Africa               Cassava    638
1             Africa                 Maize    814
2             Africa  Plantains and others    242
3             Africa              Potatoes    660
4             Africa           Rice, paddy    682
5             Africa               Sorghum    682
6             Africa              Soybeans    330
7             Africa        Sweet potatoes    594
8             Africa                 Wheat    506
9             Africa                  Yams    352
10              Asia               Cassava    264
11              Asia                 Maize    748
12              Asia  Plantains and others     44
13              Asia              Potatoes    770
14              Asia           Rice, paddy    550
15              Asia               Sorghum    352
16              Asia              Soybeans    440
17              Asia        Sweet potatoes    396
18              Asia                 Wheat    638


check the rows that do have only records for year range 1992-2013

In [136]:

file_path = 'processed_data/combined_raw_data.csv'
df = pd.read_csv(file_path)

# column_to_check = 'avg_temp'
column_to_check = 'average_rain_fall_mm_per_year'

df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

rows_to_delete = df[(df['Year'] >= 1992) & (df['Year'] <= 2013) & (df[column_to_check].isnull())].index

df.drop(rows_to_delete, inplace=True)

df.to_csv(file_path, index=False)


fill in the values where there are single missing values for average_rain_fall_mm_per_year

In [135]:
def fill_empty_with_mean(csv_file_path): 
    df = pd.read_csv(csv_file_path)
 
    for i in range(1, len(df) - 1):
        if pd.isna(df.at[i, 'average_rain_fall_mm_per_year']): 
            mean_value = (df.at[i-1, 'average_rain_fall_mm_per_year'] + df.at[i+1, 'average_rain_fall_mm_per_year']) / 2
 
            df.at[i, 'average_rain_fall_mm_per_year'] = mean_value
 
    df.to_csv(csv_file_path, index=False)
 
fill_empty_with_mean('processed_data/combined_raw_data.csv')

managing final column operations(delete, rename, reordering)

In [137]:
file_path = 'processed_data/combined_raw_data.csv'

rename_columns = {
    # 'old_column_name': 'new_column_name',
    'Area': 'Country', 
    'Item_x': "Crop",
    'Value_x': 'Quantity(hg/ha)',
    'Country_Continent': 'Continent',
    'Value_y': 'Pesticide use(toai)',
    'average_rain_fall_mm_per_year': 'Rainfall(mm)',
    'avg_temp': 'Temperature(C)', 
}
 
reorder_columns = ['Continent', 'Country', 'Year', 'Temperature(C)', 'Rainfall(mm)', 'Pesticide use(toai)', 'Crop', 'Quantity(hg/ha)']
 
# delete_columns = ['column_to_delete1', 'column_to_delete2', 'column_to_delete3']

df = pd.read_csv(file_path)

df.rename(columns=rename_columns, inplace=True)
 
df = df[reorder_columns]
 
# df.drop(columns=delete_columns, inplace=True, errors='ignore')
 
df.to_csv(file_path, index=False)
